# try 1

## import

In [2]:
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.utils import shuffle
from scipy.sparse import *
# import rec_util
pd.options.display.float_format = '{:.2f}'.format
import pandas as pd

## 파일 로드

In [77]:
raw_tr = pd.read_json("./raw_data/train.json")
raw_te = pd.read_json("./raw_data/val.json")

## 태그별 빈도 확인

In [78]:
ret = []
for tag in raw_tr.tags.tolist():
    ret += tag
from collections import Counter
r = dict(Counter(ret))

In [179]:
r = sorted(r.items(), key=lambda x: -x[1])
r[:5]

[('기분전환', 16465), ('감성', 11417), ('휴식', 11215), ('발라드', 10796), ('잔잔한', 10218)]

In [180]:
top_tags = [x[0] for x in r[:1000]]

# 태그와 노래 목록 만으로 나머지 예측하기

In [153]:
train_songs = raw_tr['songs'].tolist()
test_songs = raw_te['songs'].apply(lambda x : [float(i)for i in x]).tolist()
train_tags = raw_tr['tags'].tolist()
test_tags = raw_te['tags'].tolist()
test_ids = raw_te['id'].tolist()

### train 데이터 list to list 로 만들기

In [154]:
from itertools import groupby
tr = []
iid_to_idx = {}
idx = 0

for i, l in enumerate(train_songs):
    view = l
    for item_id in view:
        if item_id not in iid_to_idx:
            iid_to_idx[item_id] = idx
            idx += 1
    view = [iid_to_idx[x] for x in view]
    tr.append(view)

In [155]:
idx = 0
n_items = 615142
tag_to_idx = {} 
for i, tags in enumerate(train_tags):
    for tag in tags:
        if tag not in tag_to_idx:
            tag_to_idx[tag] = n_items + idx
            idx += 1
    tr[i].extend([tag_to_idx[x] for x in tags])
n_items = len(iid_to_idx)
n_tags = len(tag_to_idx)

### test 데이터 list to list 로 만들기

In [156]:
from s import groupby
te = []

idx = 0
for i, l in enumerate(test_songs):
    view = l
    ret = [] 
    for item_id in view:
        if item_id not in iid_to_idx:
            continue
        ret.append(iid_to_idx[item_id])
    te.append(ret)
idx = 0
for i, tags in enumerate(test_tags):
    ret = []
    for tag in tags:
        if tag not in tag_to_idx:
            continue
        ret.append(tag)
    te[i].extend([tag_to_idx[x] for x in ret])

# 데이터 쓰까

In [157]:
tr = shuffle(tr)

### 노래 및 태그 인덱싱 하기

In [185]:
idx_to_iid = {x:y for(y,x) in iid_to_idx.items()}
idx_to_tag = {(x - n_items):y for(y,x) in tag_to_idx.items()}

# sparse matrix
- 방법에 대해 고민 해봐야 함

In [22]:
def boolean_indexing(v):
    lens = np.array([len(item) for item in v])
    mask = lens[:,None] > np.arange(lens.max())
    out = np.zeros(mask.shape,dtype=int)
    out[mask] = np.concatenate(v)
    return out

In [23]:
tr_array = boolean_indexing(tr)

In [24]:
te_array = boolean_indexing(te)

In [78]:
tr_array = np.zeros([len(tr),len(max(tr,key = lambda x: len(x)))])
for i,j in enumerate(tr):
    tr_array[i][0:len(j)] = j

In [ ]:
# 노래 곡수 + 태그의 최대 숫자가 240인데 이렇게 나오면 안되고, 'n_tags + n_items'인 115071 x 644302인 어레이가 나와야 함

In [84]:
np.array([len (i) for i in tr]).max()

240

In [100]:
tr_array.astype(int)[0]

array([548271, 548272, 191482, 548273, 548274, 548275, 548276, 548277,
       548278, 548279, 548280, 548281, 548282,  73077, 477958, 548283,
       490260, 548284,     87,    572,     33,      1,    173,     45,
           20,   1840, 115158, 115643, 115104, 115072, 115244, 115116,
       115091, 116911, 615229, 615714, 615175, 615143, 615315, 615187,
       615162, 616982, 615229, 615714, 615175, 615143, 615315, 615187,
       615162, 616982,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [62]:
te_array.shape

(23015, 105)

In [64]:
n_tags + n_items

644302

## binary matrix 만들기
- https://stackoverflow.com/questions/50702988/creating-binary-matrix-of-each-unique-value-from-list-of-lists

- train list to list

In [135]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
tr_array = cv.fit_transform(tr)

# To transform to dense matrix
tr_array.todense()

matrix([[0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

- test list to list

In [112]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
te_array = cv.fit_transform(te)

# To transform to dense matrix
te_array.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

# 모델링
- 1. sparse 행렬 작업
- 2. 두 행렬을 합치고
- 3. 모델 fit 시키기

In [115]:
tr_csr = csr_matrix(tr_array, (len(tr), n_tags + n_items))
te_csr = csr_matrix(te_array, (len(te), n_tags + n_items))

In [116]:
import scipy.sparse
r = scipy.sparse.vstack([te_csr, tr_csr])
r = csr_matrix(r)

In [117]:
als_model = ALS(factors=128, regularization=0.08)
als_model.fit(r.T * 15.0)

### 노래 예측 모댈과, 태그 예측모댈에 user factor와 item factor를 분배

In [118]:
item_model = ALS(use_gpu=False)
tag_model = ALS(use_gpu=False)
item_model.user_factors = als_model.user_factors
tag_model.user_factors = als_model.user_factors

In [119]:
item_model.item_factors = als_model.item_factors[:n_items]
tag_model.item_factors = als_model.item_factors[n_items:]

In [120]:
als_model.item_factors.shape

(644302, 128)

In [121]:
als_model.item_factors[n_items:]

array([[-2.7139655e-01,  8.1092104e-02, -2.3679076e-02, ...,
        -4.6856251e-01,  5.2570064e-02,  2.4029304e-01],
       [-4.5710164e-01, -7.0614681e-02,  7.0186621e-03, ...,
         3.4151590e-01,  6.6628605e-01,  2.1835957e-01],
       [-4.2948008e-01, -1.3847241e-02, -5.4426279e-02, ...,
         2.7176657e-01,  6.3023746e-01,  2.0602919e-01],
       ...,
       [ 3.5432735e-03,  2.6923401e-04,  1.0970696e-03, ...,
         4.5748334e-03,  3.5547253e-03,  2.3294524e-03],
       [-1.9738213e-03, -6.2862844e-03, -2.1205323e-03, ...,
        -2.0258485e-03,  8.1458660e-03,  2.8332591e-03],
       [-2.4964832e-04, -8.6847757e-04, -2.2691030e-04, ...,
        -4.4478368e-04, -4.3195429e-05, -5.5013713e-04]], dtype=float32)

In [123]:
item_rec_csr = tr_csr[:, :n_items]
tag_rec_csr = tr_csr[:, n_items:]

## recommend 과정

In [130]:
item_ret = []
tag_ret = []
from tqdm.auto import tqdm
for u in tqdm(range(te_csr.shape[0])):
    item_rec = item_model.recommend(u, item_rec_csr, N=100)
    item_rec = [idx_to_iid[x[0]] for x in item_rec]
    tag_rec = tag_model.recommend(u, tag_rec_csr, N=100)
    tag_rec = [idx_to_tag[x[0]] for x in tag_rec if x[0] in idx_to_tag]
    item_ret.append(item_rec)
    tag_ret.append(tag_rec)

In [132]:
tag_model.item_factors

array([[-2.7139655e-01,  8.1092104e-02, -2.3679076e-02, ...,
        -4.6856251e-01,  5.2570064e-02,  2.4029304e-01],
       [-4.5710164e-01, -7.0614681e-02,  7.0186621e-03, ...,
         3.4151590e-01,  6.6628605e-01,  2.1835957e-01],
       [-4.2948008e-01, -1.3847241e-02, -5.4426279e-02, ...,
         2.7176657e-01,  6.3023746e-01,  2.0602919e-01],
       ...,
       [ 3.5432735e-03,  2.6923401e-04,  1.0970696e-03, ...,
         4.5748334e-03,  3.5547253e-03,  2.3294524e-03],
       [-1.9738213e-03, -6.2862844e-03, -2.1205323e-03, ...,
        -2.0258485e-03,  8.1458660e-03,  2.8332591e-03],
       [-2.4964832e-04, -8.6847757e-04, -2.2691030e-04, ...,
        -4.4478368e-04, -4.3195429e-05, -5.5013713e-04]], dtype=float32)

In [26]:
tag_model.item_factors

array([[ 3.55746090e-01, -5.69548428e-01, -5.59754260e-02, ...,
         4.83457536e-01,  1.95368275e-01,  1.14752978e-01],
       [ 1.13077201e-01,  4.99642581e-01,  1.01401411e-01, ...,
         8.70525062e-01,  5.08861303e-01,  4.67959046e-01],
       [ 1.34080434e-02,  5.13347864e-01,  1.56607270e-01, ...,
         6.01837337e-01,  4.08071131e-01,  3.32719356e-01],
       ...,
       [-5.10524551e-04,  3.28546972e-04,  8.81741638e-04, ...,
         8.72330857e-04,  1.68862971e-04,  1.18788111e-03],
       [ 9.88858286e-04,  1.02399255e-03,  3.29685328e-03, ...,
         1.42479164e-03, -1.69632386e-03, -1.65981124e-03],
       [ 9.73270435e-05,  1.62653669e-04, -2.30579590e-05, ...,
         1.98975016e-04,  2.20649337e-04, -1.06205756e-04]], dtype=float32)

## 최종 json파일 추출

In [133]:
returnval = []
for _id, rec, tag_rec in zip(test_ids, item_ret, tag_ret):
    returnval.append({
        "id": _id,
        "songs": rec[:100],
        "tags": tag_rec[:10]
    })

In [134]:
import json
with open('ret.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(returnval, ensure_ascii=False))

# try 2

In [11]:
raw_tag_song = pd.read_json('./raw_data/tag_songs.json')

In [12]:
val_w_full_tags = pd.read_json('./raw_data/val_copy.json')

In [13]:
raw_tag_song

,tags,songs,songs_num
0,,"[253222, 270262, 301533, 136007, 238347, 13384...","{'253222': 1, '270262': 1, '301533': 1, '13600..."
1,00,"[496456, 323561, 452688, 503200, 102389, 58602...","{'496456': 1, '323561': 1, '452688': 1, '50320..."
2,007,"[571824, 181423, 333856, 291812, 402460, 10929...","{'291812': 3, '359929': 3, '210978': 2, '33130..."
3,007시리즈,"[427367, 181423, 492240, 333856, 291812, 67710...","{'427367': 2, '181423': 2, '492240': 2, '33385..."
4,00s,"[367143, 35117, 387871, 651040, 551409, 544099...","{'580074': 3, '117595': 2, '176961': 2, '58477..."
...,...,...,...
29155,힙해,"[554524, 653392, 669251, 217370, 627771, 75076...","{'363570': 3, '653392': 2, '18117': 2, '271156..."
29156,힙힙힙,"[146247, 152953, 170279, 703985, 591826, 46060...","{'44113': 2, '146247': 1, '152953': 1, '170279..."
29157,힛뎀포크,"[251011, 14923, 605624, 4819, 442751, 538647, ...","{'251011': 1, '14923': 1, '605624': 1, '4819':..."
29158,힛뎀폭,"[251011, 14923, 605624, 4819, 442751, 538647, ...","{'251011': 1, '14923': 1, '605624': 1, '4819':..."


In [14]:
tag_song = raw_tag_song[['tags','songs']]
tag_song

,tags,songs
0,,"[253222, 270262, 301533, 136007, 238347, 13384..."
1,00,"[496456, 323561, 452688, 503200, 102389, 58602..."
2,007,"[571824, 181423, 333856, 291812, 402460, 10929..."
3,007시리즈,"[427367, 181423, 492240, 333856, 291812, 67710..."
4,00s,"[367143, 35117, 387871, 651040, 551409, 544099..."
...,...,...
29155,힙해,"[554524, 653392, 669251, 217370, 627771, 75076..."
29156,힙힙힙,"[146247, 152953, 170279, 703985, 591826, 46060..."
29157,힛뎀포크,"[251011, 14923, 605624, 4819, 442751, 538647, ..."
29158,힛뎀폭,"[251011, 14923, 605624, 4819, 442751, 538647, ..."


In [109]:
import itertools
a = list(itertools.islice(raw_tag_song[raw_tag_song['tags'] == "OST"]['songs_num'].tolist()[0].items(), 20))
b = list(itertools.islice(raw_tag_song[raw_tag_song['tags'] == "힙합"]['songs_num'].tolist()[0].items(), 20))

In [81]:
a = list(itertools.islice(raw_tag_song[raw_tag_song['tags'] == "힙합"]['songs_num'].tolist()[0].items(), 20000))

In [35]:
len(val_w_full_tags)

23015

In [94]:
tags_under_10 = val_w_full_tags[val_w_full_tags['tags'].apply(lambda x : len(x)) < 10]

In [113]:
tags_under_10

,tags,id,plylst_title,songs,like_cnt,updt_date
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
8,[스트레스],80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],127,2017-02-09 17:33:45.000
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000
...,...,...,...,...,...,...
22991,[],32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000
22992,[사랑],32812,♥옷차림이 가벼워질때 부담없이 듣는음악♥,[],8,2015-05-08 00:09:01.000
22996,[],86721,해 저무는 밤,[],5,2016-04-27 15:32:55.000
23002,"[텐션, 명곡, 학창시절, 텐션업]",56570,숨어듣는명곡! 텐션올릴때듣는 그시절그노래!! (그냥명곡있을수있음주의),[],538,2020-04-18 22:02:07.000


# song_count 

In [17]:
# 중복 검사, 같은 노래면 카운트를 더해라(dict로 만들어서)
def song_count(tags):
    ls = []
    for tag in tags:
        ls += list(itertools.islice(raw_tag_song[raw_tag_song['tags'] == tag]['songs_num'].tolist()[0].items(), 20))
    ls.sort(key = lambda element : element[1], reverse=True)
    
    return [int(i[0]) for i in ls]

In [18]:
raw_tag_song[raw_tag_song['tags'] == 'OST']['songs_num'].index.values[0]

2615

In [19]:
# songs에 노래가 있는 데이터
val_w_full_tags_songs_in = val_w_full_tags[val_w_full_tags['songs'].apply(lambda x : len(x)) != 0]
val_w_full_tags_songs_in

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[OST, 기분전환, 디즈니, 애니메이션, 영화, 팝, 겨울, 추억, 휴식, 힐링]",118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000
2,"[발라드, 추억, 회상, 이별, 슬픔, 설렘, 사랑, 싸이월드, 밤, 새벽]",51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000
3,"[발라드, 감성, 이별, 기분전환, 사랑, 카페, 드라이브, 잔잔한, 새벽, 휴식]",45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000
4,"[CCM, 찬양, 은혜, 사랑, 예배, 국내ccm, 위로, 은혜로운, 찬송가, 교회]",79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000
5,"[Pop, 기분전환, 드라이브, 빌보드, 빌보드Hot50, 트렌디, 스트레스, 빌보...",138538,,"[173033, 138237, 63140, 15733, 461780, 308519,...",2,2019-06-12 17:43:35.000
...,...,...,...,...,...,...
23010,"[발라드, 기분전환, 감성, 사랑, 이별, 새벽, 휴식, 잔잔한, 밤, 추억]",101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000
23011,"[힐링, 휴식, 기분전환, 감성, 잔잔한, 발라드, 새벽, 사랑, 밤, 추억]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000
23012,"[기분전환, 팝, 드라이브, 휴식, 힐링, 감성, Pop, 잔잔한, 팝송, 새벽]",77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000
23013,"[클래식, 휴식, 힐링, 잔잔한, 팝, 추억, 기분전환, 올드팝, 감성, 피아노]",36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000


In [20]:
# 8분 걸림
val_w_full_tags_songs_in['count_top_songs'] = val_w_full_tags_songs_in['tags'].apply(lambda x : song_count(x))

In [26]:
# count_top_songs len 100인지 확인
val_w_full_tags_songs_in

,tags,id,plylst_title,songs,like_cnt,updt_date,count_top_songs
0,"[OST, 기분전환, 디즈니, 애니메이션, 영화, 팝, 겨울, 추억, 휴식, 힐링]",118598,,"[373313, 151080, 275346, 696876, 165237, 52593...",1675,2019-05-27 14:14:33.000,"[463173, 648628, 422915, 520093, 349492, 35030..."
2,"[발라드, 추억, 회상, 이별, 슬픔, 설렘, 사랑, 싸이월드, 밤, 새벽]",51464,,"[529437, 516103, 360067, 705713, 226062, 37089...",62,2008-06-21 23:26:22.000,"[680366, 215411, 133143, 463173, 349492, 42772..."
3,"[발라드, 감성, 이별, 기분전환, 사랑, 카페, 드라이브, 잔잔한, 새벽, 휴식]",45144,,"[589668, 21711, 570151, 320043, 13930, 599327,...",20,2017-10-30 18:15:43.000,"[519391, 640657, 232606, 698536, 519391, 64065..."
4,"[CCM, 찬양, 은혜, 사랑, 예배, 국내ccm, 위로, 은혜로운, 찬송가, 교회]",79929,,"[672718, 121924, 102694, 683657, 201558, 38511...",20,2017-02-07 11:40:42.000,"[133143, 610933, 348200, 349492, 701557, 58225..."
5,"[Pop, 기분전환, 드라이브, 빌보드, 빌보드Hot50, 트렌디, 스트레스, 빌보...",138538,,"[173033, 138237, 63140, 15733, 461780, 308519,...",2,2019-06-12 17:43:35.000,"[519391, 640657, 698536, 133484, 232606, 11228..."
...,...,...,...,...,...,...,...
23010,"[발라드, 기분전환, 감성, 사랑, 이별, 새벽, 휴식, 잔잔한, 밤, 추억]",101722,,"[75842, 26083, 244183, 684715, 500593, 508608,...",17,2015-12-17 14:06:05.000,"[357367, 144663, 519391, 680366, 640657, 46317..."
23011,"[힐링, 휴식, 기분전환, 감성, 잔잔한, 발라드, 새벽, 사랑, 밤, 추억]",122127,,"[450275, 487671, 561031, 663944, 628672, 59121...",10,2020-04-16 21:35:44.000,"[357367, 144663, 519391, 680366, 640657, 46317..."
23012,"[기분전환, 팝, 드라이브, 휴식, 힐링, 감성, Pop, 잔잔한, 팝송, 새벽]",77438,,"[625875, 464051, 11657, 236393, 358186, 213435...",0,2019-03-27 15:27:40.000,"[519391, 640657, 357367, 698536, 133484, 23260..."
23013,"[클래식, 휴식, 힐링, 잔잔한, 팝, 추억, 기분전환, 올드팝, 감성, 피아노]",36231,,"[161094, 665833, 688145, 432735, 439938, 12665...",31,2015-11-18 11:49:09.000,"[357367, 144663, 519391, 640657, 463173, 34949..."


In [31]:
val_w_full_tags_songs_in[val_w_full_tags_songs_in['count_top_songs'].apply(lambda x : len(x)) != 100]

,tags,id,plylst_title,songs,like_cnt,updt_date,count_top_songs
60,[],47555,,"[136761, 507344]",0,2015-11-18 14:12:07.000,[]
151,[재즈],44792,,"[1222, 78622, 696595, 307959, 411063, 587256, ...",15,2017-06-26 15:00:47.000,"[92908, 705515, 321724, 650367, 356141, 222610..."
233,[],130185,,"[537913, 577336, 10370, 299500, 619981, 671353...",1,2019-08-25 14:49:24.000,[]
251,"[히스토리컬클래식, 위대한클래식, 여행, 드라이브, 산책, 힐링음악, 상처입은사람에...",123715,,"[208601, 641422, 503734, 132321, 433253, 33463...",43,2018-08-20 15:39:15.000,"[519391, 640657, 698536, 133484, 232606, 11228..."
465,"[이니셜d, 사고조심, 슈퍼유로비트, 드라이브]",67885,,"[486175, 26894, 4478, 423862, 231215, 288324, ...",130,2020-03-19 11:48:35.000,"[519391, 640657, 698536, 133484, 232606, 11228..."
...,...,...,...,...,...,...,...
22356,[],144555,,"[362641, 614449, 177918, 310240, 495650]",1,2015-11-09 14:13:20.000,[]
22397,"[도서관, 시험기간, 공부할때, 힐링, 휴식, 잔잔한]",122260,,"[581306, 331503, 24280, 395699, 115906, 357853...",9,2018-01-18 23:10:42.000,"[349492, 680366, 215411, 463173, 648628, 39682..."
22776,"[일본노래, 시티팝, JPOP, 일본, J_POP]",43959,,"[563916, 216457, 416008, 189563, 266866, 40061...",0,2019-10-10 13:13:31.000,"[632189, 204900, 580527, 624957, 608666, 60891..."
22803,"[클럽, 일렉, 드라이브, 매장음악, 댄스]",78309,,"[351621, 246673, 102602, 305054, 355977, 63160...",3,2014-01-20 12:29:48.000,"[519391, 640657, 698536, 133484, 232606, 11228..."


In [14]:
# songs에 노래가 없는 데이터
val_w_full_tags_not_in = val_w_full_tags[val_w_full_tags['songs'].apply(lambda x : len(x)) == 0]
val_w_full_tags_not_in

,tags,id,plylst_title,songs,like_cnt,updt_date
1,[],131447,앨리스테이블,[],1,2014-07-16 15:24:24.000
8,[스트레스],80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],127,2017-02-09 17:33:45.000
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000
20,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000
...,...,...,...,...,...,...
22991,[],32537,컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작,[],28,2019-06-17 14:22:48.000
22992,[사랑],32812,♥옷차림이 가벼워질때 부담없이 듣는음악♥,[],8,2015-05-08 00:09:01.000
22996,[],86721,해 저무는 밤,[],5,2016-04-27 15:32:55.000
23002,"[텐션, 명곡, 학창시절, 텐션업]",56570,숨어듣는명곡! 텐션올릴때듣는 그시절그노래!! (그냥명곡있을수있음주의),[],538,2020-04-18 22:02:07.000


In [15]:
# plylst_title이 없는것
val_w_full_tags_not_in[val_w_full_tags_not_in['plylst_title'].apply(lambda x : len(x)) == 0]

,tags,id,plylst_title,songs,like_cnt,updt_date
4514,[알앤비],120699,,[],3,2016-02-04 16:39:50.000
6113,[],108253,,[],28,2012-07-27 17:22:57.000
12607,[],2996,,[],229,2018-10-08 13:16:50.000
17558,[],125531,,[],2,2014-06-09 14:48:35.000
19221,"[봄, 연주곡, Morning, Instrumental, 아침]",15941,,[],13,2019-05-24 09:52:50.000
22830,[],22014,,[],3,2015-05-25 17:10:52.000


## 형태소 분석

In [226]:
# 1분
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

t = Okt()
vectorizer = CountVectorizer(min_df = 1)

contents_tokens = [t.nouns(each) for each in raw_tr['tags'].apply(lambda x : " ".join(x))]

In [227]:
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + " " + word
    
    contents_for_vectorize.append(sentence)
    
len(contents_for_vectorize)

115071

In [228]:
contents_for_vectorize

[' 락',
 ' 추억 회상',
 ' 까페',
 ' 연말 눈 날 캐럴 분위기 크리스마스캐럴 겨울 노래 크리스마스 겨울왕국 크리스마스 송',
 ' 댄스',
 ' 운동 드라이브 트로피 컬 하우스 힐링 기분 전환 팝 트렌드 렉',
 ' 짝사랑 취향 저격 슬픔 고백 사랑 이별',
 ' 추억 회상',
 ' 일렉트로니카 포크 메탈 락 댄스',
 ' 록 이일우 에센셜 메탈 락',
 ' 댄스 걸그룹댄스 스트레스 해소',
 ' 새해 여행 프로필 음악 카카오 톡 기분 전환 소원 프로필 소망 다짐 카톡',
 ' 우울 힐링 거',
 ' 힙합 밤 새벽 감각 드라이브 국내 그루브',
 ' 가을 재즈',
 ' 락',
 ' 감성 질리 나',
 ' 봄 사랑',
 ' 비 날 누군가 때',
 ' 스밍 목록 폐막식 올림픽 엑소',
 ' 혼자 새벽 감성 고민 맥주 한잔',
 ' 카페 재즈',
 '',
 ' 댄스',
 ' 감성 어쿠스틱 새벽 밤',
 ' 밤 새벽',
 ' 힐링 드라이브 에너지 여행',
 ' 듀엣 취향 저격 달달 피쳐링 사랑 남녀',
 ' 까페 사랑',
 ' 힙합 신인 국힙',
 ' 거슈윈 존 윌리엄스 랩소디 블루 미국 클래식 영화 클래식 아메리칸 클래식',
 ' 겨울 기분 전환',
 ' 아침 빅밴드 재즈 스윙 스윙재즈',
 ' 휴식 힐링 사랑',
 ' 여름 일렉트로니카 매장 음악 취향 저격 댄스 드라이브 여행 기분 전환 렉',
 ' 까페 사랑',
 ' 사랑',
 ' 헬스 스포츠 피트니스 운동 다이어트 런닝 레깅스 필라테스 산책 요가',
 ' 발라드',
 ' 겨울 크리스마스 캐롤',
 ' 위로 힐링',
 ' 기분 전환 가을',
 ' 감성 올드송 추억 휴식 옛날 노래',
 ' 월드 연주 재즈',
 ' 밤 새벽',
 ' 슬픔 밤 이별 새벽',
 ' 클럽 스트레스',
 ' 휴식 주말 오후',
 ' 힐링 휴식 운동',
 ' 드라이브',
 ' 여행 드라이브 산책',
 ' 나 나 얌',
 ' 중국 음악 광동 홍콩 대만 중국 노래 중국 중음',
 ' 휴식 힐링',
 ' 포크 내 음악 트랙 노래 추

In [249]:
# 벡터라이즈
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 1, decode_error = 'ignore')
X = vectorizer.fit_transform(contents_for_vectorize)
X

<115071x10395 sparse matrix of type '<class 'numpy.int64'>'
	with 395059 stored elements in Compressed Sparse Row format>

In [ ]:
# 메모리 터짐
from sklearn.metrics import pairwise_distances
pairwise_matrix = pairwise_distances(X, X, n_jobs=-1)

# 유사도 측정할 테스트 문장

In [29]:
val_w_full_tags_not_in['plylst_title']

1                                        앨리스테이블
8                 리듬타면서 빡시게 운동하자!!!(스트레스 날리자):}
9                       기분 좋은 재즈와 함께 만드는 달달한 하루
17                                 다시 생각나는 그 사람
20                            불면증엔 아름다운 뉴에이지 곡들
                          ...                  
22991         컨트리 황제 조니 캐시가 선 레코드 시절 발표한 초기 대표작
22992                    ♥옷차림이 가벼워질때 부담없이 듣는음악♥
22996                                   해 저무는 밤
23002    숨어듣는명곡! 텐션올릴때듣는 그시절그노래!! (그냥명곡있을수있음주의)
23008                    [별책부록] Sweetmind의 음악마중
Name: plylst_title, Length: 4379, dtype: object

In [30]:
new_post = [val_w_full_tags_not_in['plylst_title'].iloc[3]]
new_post_tokens = [t.nouns(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
    
    new_post_for_vectorize.append(sentence)
    
new_post_for_vectorize

[' 다시 그 사람']

In [31]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)

In [36]:
def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    
    delta = v1_normalized - v2_normalized
    
    return sp.linalg.norm(delta.toarray())

In [188]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

## 거리 구하고

In [223]:
dist = [dist_norm(each, new_post_vec) for each in X]

In [224]:
ls = []
for i in sorted(dist):
    ls.append(dist.index(i))

In [225]:
j = []
for i in raw_tr.iloc[ls].drop_duplicates('id')['tags']:
    j.append(i)
k = sum(j, [])[:11]
k

['다시',
 '고마웠어',
 '그사람',
 '볼수없는',
 '언제나좋은',
 '기분전환',
 'Pop',
 '다시듣게되는',
 '겨울to봄',
 '봄',
 '다시_봄']

In [53]:
print('Best post is ', dist.index(min(dist)), ", dist = ", min(dist))
print('Test post is --> ', new_post)
raw_tr.iloc[dist.index(min(dist))]['tags']

Best post is  8808 , dist =  0.7506618237326745
Test post is -->  ['다시 생각나는 그 사람']


['다시']

In [118]:
import operator
dist_dict = dict(enumerate(dist))
sorted_dict = sorted(dist_dict.items(), key=operator.itemgetter(1))

In [123]:
print('Best 5 post is ', sorted_dict[:10])
print('Test post is --> ', new_post)

Best 5 post is  [(395, 0.0), (722, 0.0), (1049, 0.0), (1113, 0.0), (1203, 0.0), (3561, 0.0), (4357, 0.0), (5017, 0.0), (5564, 0.0), (5947, 0.0)]
Test post is -->  ['리듬타면서 빡시게 운동하자!!!(스트레스 날리자):}']


In [120]:
raw_tr.iloc[395]['tags']

['스트레스', '운동']

In [121]:
raw_tr.iloc[722]['tags']

['스트레스', '운동']

In [124]:
raw_tr.iloc[5947]['tags']

['스트레스', '운동']

In [117]:
dist.index(min(dist))

395

In [568]:
val_w_full_tags['tags'][1]

[]

In [572]:
raw_tag_song[raw_tag_song['tags'].isna()]

,tags,songs,songs_num


In [567]:
a = song_count(val_w_full_tags['tags'][1])
a

[]

In [540]:
val_w_full_tags['tags'][12345]

['랩', '힙구']

In [209]:
val_test = val_w_full_tags[(val_w_full_tags['plylst_title'] == '')]

# apply 함수 적용

In [339]:
def count_song_id(tags): 
    return raw_tag_song[raw_tag_song['tags'] == tags]['songs'].tolist()

In [327]:
a = val_w_full_tags['tags'].apply(lambda x : [{"playlist_id": x , "tags": i, "song_id": count_song_id(i) } for i in x])

KeyboardInterrupt: 

In [340]:
a = val_w_full_tags['tags'][:5].apply(lambda x : [{"tags": i, "song_id": count_song_id(i) } for i in x])

In [358]:
a

0    [{'tags': 'OST', 'song_id': [[537468, 234710, ...
1                                                   []
2    [{'tags': '발라드', 'song_id': [[610133, 55358, 6...
3    [{'tags': '발라드', 'song_id': [[610133, 55358, 6...
4    [{'tags': 'CCM', 'song_id': [[297140, 388686, ...
Name: tags, dtype: object

In [300]:
val_test['tags'][0].index

<function list.index(value, start=0, stop=9223372036854775807, /)>

In [280]:
val_test['tags'][0]

['OST', '기분전환', '디즈니', '애니메이션', '영화', '팝', '겨울', '추억', '휴식', '힐링']

In [271]:
a

0    [{'playlist_id': 1, 'tags': 'OST', 'song_id': ...
2    [{'playlist_id': 1, 'tags': '발라드', 'song_id': ...
3    [{'playlist_id': 1, 'tags': '발라드', 'song_id': ...
4    [{'playlist_id': 1, 'tags': 'CCM', 'song_id': ...
5    [{'playlist_id': 1, 'tags': 'Pop', 'song_id': ...
Name: tags, dtype: object

In [238]:
flat_list = []
for sublist in a:
    for items in sublist:
        for item in items:
            flat_list.append(item)

In [232]:
for i in range(10):
    print(len(flat_list[i][0]))

TypeError: object of type 'int' has no len()

In [136]:
flat_list = [item for sublist in a[0][0].values.tolist() for item in sublist]
len(flat_list)

112445

In [208]:
flat_list[0]

[537468,
 234710,
 691961,
 116590,
 570884,
 632011,
 425733,
 530220,
 171609,
 303681,
 160812,
 312129,
 552735,
 393313,
 538341,
 312732,
 209779,
 429703,
 452316,
 148785,
 399636,
 356564,
 333768,
 672355,
 262019,
 299789,
 150653,
 10232,
 572932,
 514134,
 75118,
 598294,
 170159,
 393436,
 552496,
 535097,
 469459,
 16634,
 54614,
 299107,
 35603,
 378146,
 318996,
 446875,
 301977,
 686043,
 94809,
 623183,
 468612,
 341556,
 502749,
 266820,
 66897,
 46046,
 506883,
 520379,
 348895,
 386218,
 176424,
 464140,
 642481,
 542099,
 549149,
 120974,
 30020,
 32690,
 405206,
 563262,
 630884,
 121860,
 541367,
 490217,
 526056,
 654757,
 593791,
 152809,
 435392,
 574790,
 434140,
 223008,
 63272,
 540992,
 376393,
 121204,
 615111,
 366893,
 216796,
 599183,
 278062,
 629718,
 588475,
 241694,
 285794,
 472864,
 62874,
 430222,
 198674,
 97396,
 158164,
 697803,
 285282,
 151246,
 686245,
 170820,
 92,
 580748,
 315279,
 479111,
 675607,
 196994,
 408117,
 549117,
 268506,


In [122]:
a[0][0].values.tolist()

[[537468,
  234710,
  691961,
  116590,
  570884,
  632011,
  425733,
  530220,
  171609,
  303681,
  160812,
  312129,
  552735,
  393313,
  538341,
  312732,
  209779,
  429703,
  452316,
  148785,
  399636,
  356564,
  333768,
  672355,
  262019,
  299789,
  150653,
  10232,
  572932,
  514134,
  75118,
  598294,
  170159,
  393436,
  552496,
  535097,
  469459,
  16634,
  54614,
  299107,
  35603,
  378146,
  318996,
  446875,
  301977,
  686043,
  94809,
  623183,
  468612,
  341556,
  502749,
  266820,
  66897,
  46046,
  506883,
  520379,
  348895,
  386218,
  176424,
  464140,
  642481,
  542099,
  549149,
  120974,
  30020,
  32690,
  405206,
  563262,
  630884,
  121860,
  541367,
  490217,
  526056,
  654757,
  593791,
  152809,
  435392,
  574790,
  434140,
  223008,
  63272,
  540992,
  376393,
  121204,
  615111,
  366893,
  216796,
  599183,
  278062,
  629718,
  588475,
  241694,
  285794,
  472864,
  62874,
  430222,
  198674,
  97396,
  158164,
  697803,
  285282,
  1

0    [[[537468, 234710, 691961, 116590, 570884, 632...
Name: tags, dtype: object

In [104]:
a

0    [[[537468, 234710, 691961, 116590, 570884, 632...
Name: tags, dtype: object

In [55]:
tag_song[tag_song['tags'] == 'OST']['songs']

2615    [537468, 234710, 691961, 116590, 570884, 63201...
Name: songs, dtype: object

In [40]:
df = pd.Series(sum([item for item in tag_song['songs'][1:2]], [])).value_counts()

In [41]:
df

131071    1
140638    1
54875     1
662362    1
452185    1
         ..
2726      1
271883    1
77813     1
419490    1
521216    1
Length: 167, dtype: int64

In [100]:
tag_song['songs'][0].astype(str)

AttributeError: 'list' object has no attribute 'astype'

In [27]:
tag_song.iloc[:3]['tags']

0       
1     00
2    007
Name: tags, dtype: object

In [86]:
from collections import Counter
len(tag_song['songs'][0])

913408

In [92]:
frequency_list

NameError: name 'frequency_list' is not defined

In [46]:
frequency = {}
for i in tag_song[:3].index:
    tags = tag_song.get_value(i,'tags')
    songs = tag_song.get_value(i,'songs')
    for word in songs:
        count = frequency.get(word,0)
        frequency[tags] = count + 1

In [47]:
frequency

{'': 1, '00': 1, '007': 1}

In [18]:
for i in tag_song:
    print(i)

tags
songs


In [14]:
frequency

{'tags': 1, 'songs': 2}